# プロンプトを自動生成してみよう

`PromptCreator` は、目的と背景を入力として、プロンプトを自動生成するプロンプトです。

PromptGenでは、プロンプトのモデルそのものも入出力パラメータとして扱うことができます。これにより、プロンプトを自動生成するプロンプトを作成することができます。

In [1]:
import promptgen as pg

In [2]:
collection = pg.PromptCollection(load_predefined=True)
print(collection)

<PromptCollection>
Number of prompts: 6

Prompts:
- TextCategorizer: (text: str, categories: list) -> (category: str, found: bool)
- TextSummarizer: (text: str) -> (summary: str)
- PromptExampleCreator: (prompt: Prompt, n: int) -> (examples: list)
- PromptCreator: (purpose: str, background: str) -> (prompt: Prompt)
- PromptOptimizer: (original_prompt: Prompt, background: str) -> (optimized_prompt: Prompt)
- PythonCodeGenerator: (task: str) -> (reason: str, code: str)



## プロンプトを生成するプロンプト

`PromptCreator` は、プロンプトを生成するプロンプトです。

入力パラメータは、以下の通りです。

```python
class PromptCreatorInput(InputValue):
    purpose: str
    background: str
```

出力パラメータは、以下の通りです。

```python
class PromptCreatorOutput(OutputValue):
    prompt: Prompt
```

In [3]:
collection

<PromptCollection>
Number of prompts: 6

Prompts:
- TextCategorizer: (text: str, categories: list) -> (category: str, found: bool)
- TextSummarizer: (text: str) -> (summary: str)
- PromptExampleCreator: (prompt: Prompt, n: int) -> (examples: list)
- PromptCreator: (purpose: str, background: str) -> (prompt: Prompt)
- PromptOptimizer: (original_prompt: Prompt, background: str) -> (optimized_prompt: Prompt)
- PythonCodeGenerator: (task: str) -> (reason: str, code: str)

In [4]:
prompt_creator = collection['PromptCreator']
print(prompt_creator)

PromptCreator: (purpose: str, background: str) -> (prompt: Prompt)


In [5]:
prompt_creator

Prompt: PromptCreator

Create a prompt from the given purpose. Don't create an example with the input purpose. Instead, create an example with a different purpose. Consider background information that is necessary to understand the purpose.

Input Parameters:
    - purpose (str): purpose of the prompt
    - background (str): background of the prompt

Output Parameters:
    - prompt (Prompt): prompt created from the given purpose. Is has 'name', 'description', 'input_parameters', 'output_parameters', 'template', and 'examples'.

プロンプトを文字列にフォーマットすると、以下のようになります。

In [6]:
formatter = pg.KeyValuePromptFormatter()

print(formatter.format_prompt_without_input(prompt=prompt_creator))

You are an AI named "PromptCreator".
Create a prompt from the given purpose. Don't create an example with the input purpose. Instead, create an example with a different purpose. Consider background information that is necessary to understand the purpose.
You should follow 'Template' format. The format is 'key: value'.

Input Parameters:
  - purpose: purpose of the prompt
  - background: background of the prompt

Output Parameters:
  - prompt: prompt created from the given purpose. Is has 'name', 'description', 'input_parameters', 'output_parameters', 'template', and 'examples'.

Template:
Input:
purpose: """purpose of the prompt"""
background: """background of the prompt"""
Output:
prompt: { 'name': 'sample-new prompt',
  'description': 'A sample prompt.',
  'input_parameters': [{'name': 'input1', 'description': 'The first input parameter.'}, {'name': 'input2', 'description': 'The second input parameter.'}],
  'output_parameters': [{'name': 'output1', 'description': 'The first output p

## 大規模言語モデルの準備

今回は、 `gpt-3.5-turbo` を使います。

In [7]:
import openai
import os

from dotenv import load_dotenv
load_dotenv('../../.env')

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_ID")

def generate_llm_response(prompt: str, model: str) -> str:
    resp = openai.ChatCompletion.create(model=model, messages=[
        {'role': 'user', 'content': prompt}
    ], max_tokens=2048)
    return resp['choices'][0]['message'].get('content', '')

In [8]:
class PromptCreatorInput(pg.InputValue):
    purpose: str
    background: str

class PromptCreatorOutput(pg.OutputValue):
    prompt: pg.Prompt

In [14]:
class TaskSplitterInput(pg.InputValue):
    task: str

class Subtask(pg.DataClass):
    name: str
    description: str
    difficulty: int

class TaskSplitterOutput(pg.OutputValue):
    subtasks: list[Subtask]



In [10]:
raw_req = formatter.format_prompt(prompt_creator, PromptCreatorInput(
    purpose='Split a task into subtasks',
    background='style: (task: str) -> (reason: str, subtasks: list[Subtask]), where Subtask is a dict with keys (name: str, description: str, difficulty: int)')
)
raw_resp = generate_llm_response(raw_req, 'gpt-3.5-turbo')
resp = PromptCreatorOutput.from_dataclass(formatter.parse(prompt_creator, raw_resp))
task_splitter = pg.Prompt.from_dict(resp.prompt)

task_splitter

Prompt: TaskSplitter

Split a task into subtasks

Input Parameters:
    - task (str): The task to be split into subtasks

Output Parameters:
    - reason (str): Reason for splitting the task
    - subtasks (list): List of subtasks with their name, description and difficulty

作成したプロンプトを使って、タスクを分解してみましょう。

In [11]:
raw_req = formatter.format_prompt(task_splitter, pg.InputValue.from_dict({
    'task': 'Write a book'
}))
raw_resp = generate_llm_response(raw_req, 'gpt-3.5-turbo')

print(raw_resp)

reason: """Breaking down complex tasks into smaller subtasks makes them more manageable and helps improve productivity"""
subtasks: [{'name': 'Outline', 'description': 'Create a rough outline of the book with chapter titles and main ideas', 'difficulty': 2}, {'name': 'Research', 'description': 'Gather information and data to support the book’s main ideas', 'difficulty': 3}, {'name': 'Write first draft', 'description': 'Write the first draft of the book from beginning to end', 'difficulty': 5}, {'name': 'Revise', 'description': 'Edit and revise the book multiple times to improve its readability and overall quality', 'difficulty': 4}, {'name': 'Proofread and finalize', 'description': 'Check for spelling and grammar errors and finalize the book', 'difficulty': 2}]


In [12]:
output_value = TaskSplitterOutput.from_dataclass(formatter.parse(task_splitter, raw_resp))
output_value

TaskSplitterOutput(subtasks=[{'name': 'Outline', 'description': 'Create a rough outline of the book with chapter titles and main ideas', 'difficulty': 2}, {'name': 'Research', 'description': 'Gather information and data to support the book’s main ideas', 'difficulty': 3}, {'name': 'Write first draft', 'description': 'Write the first draft of the book from beginning to end', 'difficulty': 5}, {'name': 'Revise', 'description': 'Edit and revise the book multiple times to improve its readability and overall quality', 'difficulty': 4}, {'name': 'Proofread and finalize', 'description': 'Check for spelling and grammar errors and finalize the book', 'difficulty': 2}], reason='Breaking down complex tasks into smaller subtasks makes them more manageable and helps improve productivity')

In [13]:
for subtask in output_value.subtasks:
    print(f'{subtask.name}: {subtask.description}, difficulty: {subtask.difficulty}')

AttributeError: 'dict' object has no attribute 'name'

In [ ]:
type(output_value.subtasks[0])